# Optimisation of coal market with Scipy, using same principles as power market

2024-09-15



In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

pd.options.plotting.backend = "plotly"

In [12]:
from scipy.optimize import minimize, Bounds, LinearConstraint

In [13]:
np.set_printoptions(linewidth=200)

## 1. Brute-force minimisation with Scipy of total cost given the constraints

As an approximation to the Nash equilibrium (though minimisation does not guarantee NE).

List of constraints:
- mine volumes: between 0 and 4, need to sum by rows
- plant volumes: between 0 and 8, need to sum by columns
- prices: above the mine cash costs, reproduce by rows
- demand: all volume must sum to total demand

Tutorial: https://docs.scipy.org/doc/scipy/tutorial/optimize.html#constrained-minimization-of-multivariate-scalar-functions-minimize

In [14]:
def create_distance_matrix(v, w):
    M = len(v)
    N = len(w)
    return np.array([[np.linalg.norm(v[i] - w[j]) for j in range(N)] for i in range(M)])

In [15]:
def total_cost(mat_volumes, mat_prices, mat_freight):
    return np.sum(mat_volumes * mat_prices) + np.sum(mat_volumes * mat_freight)


def total_cost_squeezed(x, mat_freight):
    mat_volumes = np.reshape(x[:N*N], (N, N))
    mat_prices = np.reshape(x[N*N:2*N*N], (N, N))
    return total_cost(mat_volumes, mat_prices, mat_freight)

In [16]:
def build_constraints(N, vec_mine_costs, TOT_DEMAND, vec_mine_cap, vec_plant_cap):
    '''
    Build constraints for the optimization problem
    NEXT STEP: generalise for unequal mine and plant numbers
    '''
    no_volume_constraint = [0] * N*N
    no_price_constraint = [0] * N*N

    constraints = []
    lows = []
    highs = []

    # mine capacity constraints
    for i in range(N):
        cons = [0] * N * i + [1] * N + [0] * N * (N - i - 1)
        constraints.append(cons + no_price_constraint)
        lows.append(0)
        highs.append(vec_mine_cap[i])

    # individual mine volume constraints
    for i in range(N):
       for j in range(N):
            cons = [0] * N * i + ([0]*j + [1] + [0]*(N-j-1)) + [0] * N * (N - i - 1)
            constraints.append(cons + no_price_constraint)
            lows.append(0)
            highs.append(vec_mine_cap[i])

    # plant capacity constraints
    for i in range(N):
        cons = ([0] * i + [1] + [0] * (N - i - 1)) * 3
        constraints.append(cons + no_price_constraint)
        lows.append(0)
        highs.append(vec_plant_cap[i])

    # individual plant capacity constraints
    for i in range(N):
        for j in range(N):
            cons = [0] * N * i + ([0]*j + [1] + [0]*(N-j-1)) + [0] * N * (N - i - 1)
            constraints.append(cons + no_price_constraint)
            lows.append(0)
            highs.append(vec_plant_cap[j])

    # mine cost constraints
    for i in range(N):
        for j in range(N):
            cons = [0] * N * i + ([0]*j + [1] + [0]*(N-j-1)) + [0] * N * (N - i - 1)
            constraints.append(no_volume_constraint + cons)
            lows.append(vec_mine_costs[i])
            highs.append(np.inf)

    # demand constraint
    cons = [1] * N*N
    constraints.append(cons + no_price_constraint)
    lows.append(TOT_DEMAND)
    highs.append(TOT_DEMAND*1.2)

    return constraints, lows, highs

In [26]:
def visualise_results_scipy(res, N, mat_freight):
    '''Show results in a simple dataframe for visual exploration'''
    dfvol = pd.DataFrame(res.x[:N*N].reshape((N, N)).round(1), columns=list('abc'))
    dfprice = pd.DataFrame(res.x[N*N:].reshape((N, N)).round(1), columns=list('abc'))
    dffreight = pd.DataFrame(mat_freight.round(1), columns=list('abc'))
    df = pd.concat([dfvol, dfprice, dffreight], axis=1, keys=['Volumes', 'Prices', 'Freight'])
    return df

Define all variables, with random IPP positions.

In [17]:
np.random.seed(42)

N = 3
UNIT_FREIGHT = 0.05
LF = 0.25
MINE_CAP = 4.0

# input vectors
vec_mine_costs = np.linspace(80, 120, N)  # np.random.normal(100, 30, N)
vec_mine_cap = np.ones_like(vec_mine_costs) * MINE_CAP
vec_plant_cap = np.ones_like(vec_mine_costs) * 8.0

# output vectors
vec_plant_lf = np.ones_like(vec_mine_costs) * LF
vec_plant_volume = vec_plant_cap * vec_plant_lf
vec_mine_volume = vec_plant_volume

# aggregated variables
TOT_DEMAND = vec_plant_volume.sum()
TOT_MINE_CAP = vec_mine_cap.sum()
TOT_PLANT_CAP = vec_plant_cap.sum()

# input matrices
loc_mines = np.random.rand(N, 2) * 500
loc_plants = np.random.rand(N, 2) * 500
mat_distances = create_distance_matrix(loc_mines, loc_plants)
mat_freight = UNIT_FREIGHT * mat_distances

# output matrices
mat_volumes = np.diag(vec_mine_volume)
mat_prices = np.diag(vec_mine_costs)

In [18]:
# test function
total_cost(mat_volumes, mat_prices, mat_freight)

666.1624739684947

In [30]:
# minimise without constraints -- bad
res = minimize(total_cost_squeezed, [*mat_volumes.flatten(), *mat_prices.flatten()], args=(mat_freight,))
res

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -102673482619.16183
        x: [-1.121e+06 -1.056e+05 ...  7.329e+03  3.281e+04]
      nit: 2
      jac: [ 1.024e+04  3.072e+03 ... -2.253e+05 -1.788e+06]
 hess_inv: [[ 1.008e+00  1.962e-03 ...  3.787e-02  3.102e-01]
            [ 1.962e-03  1.000e+00 ...  3.640e-03  2.982e-02]
            ...
            [ 3.787e-02  3.640e-03 ...  9.919e-01 -6.359e-02]
            [ 3.102e-01  2.982e-02 ... -6.359e-02  5.025e-01]]
     nfev: 2254
     njev: 118

In [31]:
visualise_results_scipy(res, N, mat_freight)

Volumes                         Prices                   Freight        \
           a          b          c        a        b        c       a     b   
0 -1120962.3  -105572.6  -112726.3  11094.2   3430.8   3663.3     8.2   8.3   
1  -230576.9 -1325452.8  -255186.7   7493.1  17759.5   8292.8    18.1   4.3   
2  -227980.8  -225517.2 -1788001.1   7408.7   7328.6  32811.0    17.9  17.7   

         
      c  
0   8.9  
1  20.1  
2  20.6

Build constraints.

In [20]:
c, l, h = build_constraints(N, vec_mine_costs, TOT_DEMAND, vec_mine_cap, vec_plant_cap)

In [21]:
np.c_[c, l, h]

array([[  1. ,   1. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   4. ],
       [  0. ,   0. ,   0. ,   1. ,   1. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   4. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   4. ],
       [  1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   4. ],
       [  0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   4. ],
       [  0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   4. ],
       [  0. ,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. , 

In [22]:
# # constraint on total traded volume
# no_price_constraint = [0] * (N*N)

# # individual mine volumes
# # generalise for any N
# mine_1 = [1] * N + [0] * N * (N-1)
# mine_2 = [0] * N + [1] * N + [0] * N * (N-2)  # generalise
# mine_3 = [0] * N * (N-1) + [1] * N

# linear_constraint = LinearConstraint(
#     [
#         [1] * (N*N) + no_price_constraint,
#         mine_1 + no_price_constraint,
#         mine_2 + no_price_constraint,
#         mine_3 + no_price_constraint,
#     ]
#     [TOT_MINE_VOLUME] + [0] * N,
#     [TOT_MINE_VOLUME] + [MINE_CAP] * N
# )


# # constraint on individual mine volumes and prices
# mine_vols_low = np.zeros(N)
# mine_vols_high = vec_mine_volume
# bounds = Bounds([0.0] * (N*N*2), [np.inf] * (N*N*2))

In [23]:
linear_constraint = LinearConstraint(c, l, h)

OptimizeWarning: 

Equality and inequality constraints are specified in the same element of the constraint list. For efficient use with this method, equality and inequality constraints should be specified in separate elements of the constraint list. 

  warn("Equality and inequality constraints are specified in the same "

In [24]:
# with constraints
res = minimize(
    total_cost_squeezed,
    [*mat_volumes.flatten(), *mat_prices.flatten()],
    args=(mat_freight,),
    constraints=linear_constraint
)

In [25]:
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 561.2848318233706
       x: [ 4.000e+00 -7.627e-08 ...  1.206e+02  1.200e+02]
     nit: 18
     jac: [ 8.819e+01  8.830e+01 ...  0.000e+00  0.000e+00]
    nfev: 296
    njev: 14

In [27]:
visualise_results_scipy(res, N, mat_freight)

Volumes           Prices               Freight            
        a    b    c      a      b      c       a     b     c
0     4.0 -0.0 -0.0   80.0   80.0   80.1     8.2   8.3   8.9
1    -0.0  2.0 -0.0  100.5  100.0  101.1    18.1   4.3  20.1
2    -0.0 -0.0 -0.0  120.6  120.6  120.0    17.9  17.7  20.6

**Notes:**
- good allocation of volumes, cheapest plant 
- agreed price is not determined, clearly the derived prices are costs and not transaction prices at Nash equilibrium

## 3. Exploration of variants

### 3.1. Constant freight: show what price is

In [62]:
seed = 42
np.random.seed(seed)

N = 3
LF = 0.25
MINE_CAP = 4.0
PLANT_CAP = 8.0
UNIT_FREIGHT = 0.0001
FREIGHT_CONST = 0.0

# input vectors
vec_mine_costs = np.linspace(80, 140, N)
vec_mine_cap = np.ones_like(vec_mine_costs) * MINE_CAP
vec_plant_cap = np.ones_like(vec_mine_costs) * PLANT_CAP

# output vectors
vec_plant_lf = np.ones(N) * LF
vec_plant_volume = vec_plant_cap * vec_plant_lf
vec_mine_volume = vec_plant_volume

# aggregated variables
TOT_DEMAND = vec_plant_volume.sum()
TOT_MINE_CAP = vec_mine_cap.sum()
TOT_PLANT_CAP = vec_plant_cap.sum()

# input matrices
loc_mines = np.random.rand(N, 2) * 500
loc_plants = np.random.rand(N, 2) * 500
mat_distances = create_distance_matrix(loc_mines, loc_plants)
mat_freight = UNIT_FREIGHT * mat_distances
mat_freight = np.ones((N, N)) * FREIGHT_CONST

# output matrices
# mat_volumes = np.diag(vec_mine_volume)
# mat_prices = np.diag(vec_mine_costs)
mat_volumes = np.ones((N, N)) * TOT_DEMAND / N
mat_prices = np.diag(vec_mine_costs)

In [63]:
print(f'Mean plant volume: {vec_plant_volume.mean()}')

Mean plant volume: 2.0


In [64]:
c, l, h = build_constraints(N, vec_mine_costs, TOT_DEMAND, vec_mine_cap, vec_plant_cap)
linear_constraint = LinearConstraint(c, l, h)

In [65]:
res = minimize(
    total_cost_squeezed,
    [*mat_volumes.flatten(), *mat_prices.flatten()],
    args=(mat_freight,),
    constraints=linear_constraint
)

In [66]:
visualise_results_scipy(res, N, mat_freight)

Volumes           Prices               Freight          
        a    b    c      a      b      c       a    b    c
0    -0.0  2.0  2.0   80.0   80.0   80.0     0.0  0.0  0.0
1     1.0 -0.0  1.0  110.0  110.0  110.0     0.0  0.0  0.0
2    -0.0 -0.0 -0.0  140.0  140.0  140.0     0.0  0.0  0.0

**Notes:**
- Volumes incorrect, should be degenerate on for each mine